Задание 1.<br>
Подведите результаты эксперимента в экселе по следующим данным: ab_stats.csv - Google Диск:
Стат значимо ли отличается ARPPU в двух группах ? Какие рекомендации дадите
менеджеру?

In [ ]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [ ]:
df = pd.read_csv('ab_stats.csv')

In [ ]:
df.head()

,revenue,num_purchases,purchase,ab_group,av_site visit
0,0.0,0,0,A,9.040174
1,0.0,0,0,A,4.811628
2,0.0,0,0,A,7.342623
3,0.0,0,0,A,7.744581
4,0.0,0,0,A,10.511814


revenue - выручка <br> num_purchases - количество покупок на пользователя<br> purchase - конверсия (факт покупки)<br> ab_group - группы A (контрольная) и B (тестовая)<br> av_site visit - среднее время проведенное пользователем на сайте

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23652 entries, 0 to 23651
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   revenue        23652 non-null  float64
 1   num_purchases  23652 non-null  uint16 
 2   purchase       23652 non-null  uint16 
 3   ab_group       23652 non-null  object 
 4   av_site visit  23652 non-null  float64
dtypes: float64(2), object(1), uint16(2)
memory usage: 646.9+ KB


In [ ]:
df = df.astype({'num_purchases':np.uint16})

In [ ]:
df = df.astype({'purchase':np.uint16}) # изменили тип данных в столбцах "num_purchases" и "purchase", затем перезапускаем df.info()

In [ ]:
df.describe()

,revenue,num_purchases,purchase,av_site visit
count,23652.000000,23652.000000,23652.000000,23652.000000
mean,0.324689,0.043590,0.020717,7.013112
std,9.557730,1.079403,0.142438,3.154584
min,0.000000,0.000000,0.000000,-12.073486
25%,0.000000,0.000000,0.000000,5.173787
50%,0.000000,0.000000,0.000000,7.007936
75%,0.000000,0.000000,0.000000,8.864119
max,1303.609284,152.000000,1.000000,22.446822


In [ ]:
df['ab_group'].value_counts()

ab_group
A    11835
B    11817
Name: count, dtype: int64

In [ ]:
df[df['av_site visit'] < 0].ab_group.value_counts()

ab_group
B    462
A      6
Name: count, dtype: int64

Группа В забагована, если выборка с багом меньше 5% от совокупности, то эти данные можно выкинуть - сейчас в группе В 3,9% ошибок.

In [ ]:
df = df[df['av_site visit'] > 0]
# если ошибка меньше 5%, позициями с отрицательным значением можно принебречь (т.е. просто их выкидываем), потом повторно запускаем df.describe()

In [ ]:
df.describe()

,revenue,num_purchases,purchase,av_site visit
count,23184.000000,23184.000000,23184.000000,23184.000000
mean,0.326511,0.043780,0.020704,7.188254
std,9.645839,1.089462,0.142394,2.924011
min,0.000000,0.000000,0.000000,0.008679
25%,0.000000,0.000000,0.000000,5.314379
50%,0.000000,0.000000,0.000000,7.068656
75%,0.000000,0.000000,0.000000,8.910900
max,1303.609284,152.000000,1.000000,22.446822


Рассмотрим по отдельности группы А и В

In [ ]:
df[df['ab_group']=='A'].describe()

,revenue,num_purchases,purchase,av_site visit
count,11829.000000,11829.000000,11829.000000,11829.000000
mean,0.404667,0.050723,0.021642,6.980379
std,13.136545,1.467883,0.145517,2.004450
min,0.000000,0.000000,0.000000,0.145466
25%,0.000000,0.000000,0.000000,5.657590
50%,0.000000,0.000000,0.000000,6.984166
75%,0.000000,0.000000,0.000000,8.346243
max,1303.609284,152.000000,1.000000,17.728836


In [ ]:
df[df['ab_group']=='B'].describe()

,revenue,num_purchases,purchase,av_site visit
count,11355.000000,11355.000000,11355.000000,11355.000000
mean,0.245093,0.036548,0.019727,7.404806
std,3.192299,0.422820,0.139067,3.630412
min,0.000000,0.000000,0.000000,0.008679
25%,0.000000,0.000000,0.000000,4.746728
50%,0.000000,0.000000,0.000000,7.262590
75%,0.000000,0.000000,0.000000,9.889165
max,113.830000,25.000000,1.000000,22.446822


Максимальная выручка в группе А значительно больше, чем в группе B.

Найдем показатель ARPPU <br>

ARPPU - это средняя выручка на одного платящего пользователя, говоря другими словами, это отношение дохода к числу пользователей, совершивших оплату в определённый период.

In [ ]:
df.groupby('ab_group')['revenue'].describe()

,count,mean,std,min,25%,50%,75%,max
ab_group,,,,,,,,
A,11829.0,0.404667,13.136545,0.0,0.0,0.0,0.0,1303.609284
B,11355.0,0.245093,3.192299,0.0,0.0,0.0,0.0,113.830000


Данные распределены ненормально, поэтому нужно использовать непараметрический критерий Манна-Уитни.

In [ ]:
group_a = df[df['ab_group']=='A']
group_b = df[df['ab_group']=='B']

In [ ]:
mw_stats = stats.mannwhitneyu(x=group_a.query('purchase ==1')['revenue'].values,
                                                  y=group_b.query('purchase ==1')['revenue'].values)
mw_stats

MannwhitneyuResult(statistic=28485.5, pvalue=0.902319344132039)

Статистически незначимых различий нет, т.к. pvalue=0.90, поэтому оставляем контрольную версию (вариант А).